<a href="https://colab.research.google.com/github/md-rashid/CS224W_final_project_testing/blob/main/224W_Final_Project_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Colab, we will train an implementation of Dynaformer in PyG, mostly based on the Graphormer implementation at https://github.com/leffff/graphormer-pyg/blob/main/graphormer/layers.py, with some Dynaformer specific modifications.

In [1]:
%load_ext autoreload
%autoreload 2

The files with the model and test dataset will be imported from a GitHub repository by the following cell.

In [2]:
#!git clone https://github.com/rbasto19/CS224W_final_project.git
!git clone https://github.com/md-rashid/CS224W_final_project_testing.git

Cloning into 'CS224W_final_project_testing'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 35 (delta 16), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (35/35), 1.07 MiB | 1.17 MiB/s, done.
Resolving deltas: 100% (16/16), done.


Install necessary modules.

In [3]:
!pip install torch-geometric
!pip install torch
!pip install rdkit
!pip install networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 189.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 58.6 MB/s eta 0:00:00


Import necessary modules.

In [14]:
import torch
from torch_geometric.datasets import MoleculeNet
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data
from torch.utils.data import Subset
from torch_geometric.utils.convert import to_networkx
import networkx as nx
from networkx import all_pairs_shortest_path
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
from torch_geometric.nn.pool import global_mean_pool

import CS224W_final_project_testing.layers as layers
import CS224W_final_project_testing.model as model

ModuleNotFoundError: No module named 'CS224W_final_project'

Load dataset (we start with a small dataset for testing purposes).

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Ahsan added this snippet to make the code run, otherwise device below was throwing error

with open('CS224W_final_project_testing/refined-set-2020-5-5-5_test.pkl', 'rb') as f:
  dataset = pickle.load(f)

for i in range(len(dataset)):
  dataset[i] = Data(**dataset[i].__dict__).to(device)  # allowing to use different pyg version

NameError: name 'pickle' is not defined

Ligand-binding site visualization (visualization code generated with ChatGPT)

In [ ]:
import plotly.graph_objects as go

data = dataset[0]
node_trace = go.Scatter3d(
    x=data.pos[:, 0],
    y=data.pos[:, 1],
    z=data.pos[:, 2],
    mode='markers',
    marker=dict(size=5, color='blue'),
    text=[f'Node {i}' for i in range(data.pos.size(0))]
)

# Lines for edges
edge_traces = []
for i, j in data.edge_index.t():
    edge_traces.append(go.Scatter3d(
        x=[data.pos[i, 0], data.pos[j, 0]],
        y=[data.pos[i, 1], data.pos[j, 1]],
        z=[data.pos[i, 2], data.pos[j, 2]],
        mode='lines',
        line=dict(color='black', width=2),
    ))

# Combine the traces and plot
fig = go.Figure(data=[node_trace] + edge_traces)
fig.update_layout(scene=dict(
    xaxis=dict(title='X'),
    yaxis=dict(title='Y'),
    zaxis=dict(title='Z'),
))
fig.show()

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

Create the model and split dataset. Here we are using the default parameters in the Graphormer implementation for testing purposes.

In [ ]:
graph_model = model.Graphormer(
    num_layers=4,
    input_node_dim=dataset[0].num_node_features,
    node_dim=32,
    input_edge_dim=dataset[0].num_edge_features,
    edge_dim=16,
    output_dim=1,
    n_heads=4,
    ff_dim=32,
    max_in_degree=4,
    max_out_degree=4,
    max_path_distance=4,
    num_heads_spatial=4
)
graph_model.to(device)

Graphormer(
  (node_in_lin): Linear(in_features=9, out_features=32, bias=True)
  (edge_in_lin): Linear(in_features=3, out_features=16, bias=True)
  (centrality_encoding): CentralityEncoding()
  (spatial_encoding): SpatialEncoding()
  (layers): ModuleList(
    (0-3): 4 x GraphormerEncoderLayer(
      (attention): GraphormerMultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x GraphormerAttentionHead(
            (edge_encoding): EdgeEncoding()
            (q): Linear(in_features=32, out_features=32, bias=True)
            (k): Linear(in_features=32, out_features=32, bias=True)
            (v): Linear(in_features=32, out_features=32, bias=True)
          )
        )
        (linear): Linear(in_features=128, out_features=32, bias=True)
      )
      (ln_1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
      (ff): Sequential(
        (0): Linear(in_features=32, out_features=32, bias=True)
        (1)

In [ ]:
test_ids, train_ids = train_test_split([i for i in range(len(dataset))], test_size=0.8, random_state=42)
train_loader = DataLoader(Subset(dataset, train_ids), batch_size=1)
test_loader = DataLoader(Subset(dataset, test_ids), batch_size=1)

Define optimizer, loss function, and start training the model.

In [ ]:
optimizer = torch.optim.AdamW(graph_model.parameters(), lr=3e-4)
loss_function = torch.nn.L1Loss(reduction="sum")

In [ ]:
for epoch in range(10):
    graph_model.train()
    batch_loss = 0.0
    for batch in tqdm(train_loader):
        y = batch.y
        optimizer.zero_grad()
        output = global_mean_pool(graph_model(batch), batch.batch)
        loss = loss_function(output, y)
        batch_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(graph_model.parameters(), 1.0)
        optimizer.step()
    print("TRAIN_LOSS", batch_loss / len(train_ids))

    model.eval()
    batch_loss = 0.0
    for batch in tqdm(test_loader):
        y = batch.y

        with torch.no_grad():
            output = global_mean_pool(graph_model(batch), batch.batch)
            loss = loss_function(output, y)

        batch_loss += loss.item()

    print("EVAL LOSS", batch_loss / len(test_ids))

  0%|          | 0/156 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:128: UserWarning:

Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.

  3%|▎         | 4/156 [11:04<7:00:50, 166.12s/it]


KeyboardInterrupt: 